<a id="title_ID"></a>
# Testing outlier_detection step with MIRI simulated data

<span style="color:red"> **Instruments Affected**</span>: FGS, MIRI, NIRCam, NIRISS, NIRSpec 

Tested on MIRI Simulated data

### Table of Contents
<div style="text-align: left"> 

<br>  [Introduction](#intro_ID) <br> [Imports](#imports_ID) <br> [Set up association files](#associations_ID) <br> [Insert outliers](#outliers_ID) <br> [Run Pipeline](#pipeline_ID) <br> [Check Results](#output_ID) <br> [About This Notebook](#about_ID) <br>


</div>

<a id="intro_ID"></a>
## Introduction

This notebook processes an image through calwebb_image2 and calwebb_image3 (calwebb_detector1 is optional) and examines the output table of the source_catalog step. The steps are as follow:

1) Set up data path and image list file.

2) Set up association files.

3) Create outlier pixels in input images.

4) Run outlier_detection step in calwebb_image3. 

6) Compare before and after DQ values of outlier pixels

These steps are set up with simulated MIRI images.

The pipeline documentation for this step can be found here: https://jwst-pipeline.readthedocs.io/en/latest/jwst/outlier_detection/main.html

The pipeline code is available on GitHub: https://github.com/spacetelescope/jwst

Author: T. Temim (some parts adopted from the test_outlier_detection.py code for NIRCam)

[Top of Page](#title_ID)

<a id="imports_ID"></a>
### Set up import statements

The following packages are needed to enable this notebook to run:
* astropy for coordinate handling and calculations
* jwst to run the pipeline steps and create associations
* matplotlib for plotting
* ci_watson for retrieving data from artifactory

[Top of Page](#title_ID)

In [1]:
import pytest
import numpy as np
from glob import glob
import json
import jwst
from astropy.io import fits, ascii
from astropy.coordinates import Angle
from astropy.table import Table, vstack, unique
from astropy.stats import sigma_clip
from jwst.pipeline import calwebb_image3
from jwst.associations import asn_from_list
import matplotlib.pyplot as plt
import random
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.outlier_detection import outlier_detection
from ci_watson.artifactory_helpers import get_bigdata

In [2]:
# Set environment variables
import os
os.environ['CRDS_PATH']='$HOME/crds_cache'
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'
os.environ['CRDS_CONTEXT']='jwst_0619.pmap'
os.environ['TEST_BIGDATA']='https://bytesalad.stsci.edu/artifactory/'


### Print pipeline version

In [3]:
jwst.__version__ 

'0.17.1'

### Retrieve data from Artifactory

In [4]:
input_file1 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits')
input_file2 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq2_MIRIMAGE_F560Wexp1_cal.fits')
input_file3 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq3_MIRIMAGE_F560Wexp1_cal.fits')
input_file4 = get_bigdata('jwst_validation_notebooks',
                     'validation_data',
                     'outlier_detection',
                     'outlier_detection_miri_test', 
                     'det_image_seq4_MIRIMAGE_F560Wexp1_cal.fits')

In [5]:
input_file_names=[]
input_file_names=[input_file1,input_file2,input_file3,input_file4]

imlist1=['det_image_seq1_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq2_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq3_MIRIMAGE_F560Wexp1_cal.fits','det_image_seq4_MIRIMAGE_F560Wexp1_cal.fits']

In [6]:
save_figs=False

<a id="associations_ID"></a>
### Set up association files
The level three pipeline relies on an association file to specify which files are to be combined and provide the output file name.

The level two pipeline can take individual images or an association file as input. The cell below sets up association files for both level2 and level3 files.

[Top of Page](#title_ID)

In [7]:
# use asn_from_list to create association table
cal_list=imlist1
asn = asn_from_list.asn_from_list(cal_list, rule=DMS_Level3_Base, product_name='outlier_combined.fits')

# use this if you need to add non'science' exposure types
#asn['products'][0]['members'][1]['exptype'] = 'background'
#asn['products'][0]['members'][2]['exptype'] = 'sourcecat'

# dump association table to a .json file for use in image3
with open('outlier_asnfile.json', 'w') as fp:
    fp.write(asn.dump()[1])

outlier_json_file='outlier_asnfile.json'
    
json_file = outlier_json_file
file_list = []
file_list2 = []
with open(json_file) as json_data:
    d = json.load(json_data)
    members = d['products'][0]['members']
    for item in np.arange(0,len(members)):
        file_list.append(members[item]['expname'])
        file_list2.append(members[item]['expname'][:-5]+"_outlier.fits")

    
asn2 = asn_from_list.asn_from_list(file_list2, rule=DMS_Level3_Base, product_name='outlier_combined2.fits')

# use this if you need to add non'science' exposure types
#asn['products'][0]['members'][1]['exptype'] = 'background'
#asn['products'][0]['members'][2]['exptype'] = 'sourcecat'

# dump association table to a .json file for use in image3
with open('outlier_asnfile2.json', 'w') as fp:
    fp.write(asn2.dump()[1])
    
outlier_json_file2='outlier_asnfile2.json'

<a id="outliers_ID"></a>
## Insert outliers 

Insert outliers into the data to see if they are detected in the output of the step.

### Choose random pixels to produce outliers

[Top of Page](#title_ID)

In [8]:
pixloc = []
for i in range(len(file_list)):
    pixloc.append([random.randint(20,1010),random.randint(430,1010)])
    pixloc.append([random.randint(20,1010),random.randint(430,1010)])
    pixloc.append([random.randint(20,1010),random.randint(430,1010)])
    pixloc.append([random.randint(20,1010),random.randint(430,1010)])
pixloc2 = np.array(pixloc)

### Assign values to outlier pixels

In [9]:
images = {}
for i in range(len(file_list)):
     with fits.open(input_file_names[i]) as h:
        j = 4*i
        med = np.median(h['SCI'].data)
        h['SCI'].data[:,:] = 1.0   # this line should eventually be commented out (only used as a test)
        h['SCI'].data[pixloc2[j,0],pixloc2[j,1]] = med*3.0
        h['SCI'].data[pixloc2[j+1,0],pixloc2[j+1,1]] = med*5.0
        h['SCI'].data[pixloc2[j+2,0],pixloc2[j+2,1]] = med*10.0
        h['SCI'].data[pixloc2[j+3,0],pixloc2[j+3,1]] = med*20.0
        h['DQ'].data[:,:] = 0
        h.writeto(file_list2[i],overwrite=True)
        images["img{0}".format(i)]=h


<a id="intro_ID"></a>
### Run outlier_detection step in calwebb_image3
The pipeline can be set to skip steps that are not needed or in this case, are not being tested.

[Top of Page](#title_ID)

In [10]:
im3 = calwebb_image3.Image3Pipeline()
im3.tweakreg.skip = True
im3.source_catalog.skip = True
im3.resample.skip = True
im3.skymatch.skip = True
im3.save_results=True
#im3.resample.blendheaders = False
#im3.outlier_detection.save_intermediate_results = True
im3.run(outlier_json_file2)

2021-01-04 16:03:56,198 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.


2021-01-04 16:03:56,200 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.


2021-01-04 16:03:56,202 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.


2021-01-04 16:03:56,203 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.


2021-01-04 16:03:56,205 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.


...

2021-01-04 16:04:08,259 - stpipe.Image3Pipeline.source_catalog - INFO - Step source_catalog running with args (<ModelContainer>,).


2021-01-04 16:04:08,261 - stpipe.Image3Pipeline.source_catalog - INFO - Step source_catalog parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False, 'output_use_index': True, 'save_results': True, 'skip': True, 'suffix': 'cat', 'search_output_file': True, 'input_dir': '', 'bkg_boxsize': 100, 'kernel_fwhm': 2.0, 'snr_threshold': 3.0, 'npixels': 5, 'deblend': False, 'aperture_ee1': 30, 'aperture_ee2': 50, 'aperture_ee3': 70, 'ci1_star_threshold': 2.0, 'ci2_star_threshold': 1.8}


2021-01-04 16:04:08,261 - stpipe.Image3Pipeline.source_catalog - INFO - Step skipped.


2021-01-04 16:04:08,262 - stpipe.Image3Pipeline.source_catalog - INFO - Step source_catalog done


2021-01-04 16:04:08,264 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline done


<a id="output_ID"></a>
## Check results

### Get filenames and outlier detection outputs

In [11]:
output_files = []
input_files = []
with open(outlier_json_file) as json_data:
    d = json.load(json_data)
    members = d['products'][0]['members']
for item in np.arange(0,len(members)):
    expname = members[item]['expname'][:-5]+"_outlier.fits"
    expname2 = members[item]['expname'][:-5]+"_outlier_a3001_crf.fits"
    input_files.append(expname)
    output_files.append(expname2)
    output_files.sort()

all_out_dqs = []
dq_before = []
dq_after = []    

### Get the before DQ pixel values (should be 0.0)

In [12]:
for i in range(len(file_list)):
    with fits.open(input_files[i]) as h:
        j = 4*i
        dq_before.append([pixloc2[j,:],h['DQ'].data[pixloc2[j,0],pixloc2[j,1]]])
        dq_before.append([pixloc2[j+1,:],h['DQ'].data[pixloc2[j+1,0],pixloc2[j+1,1]]])
        dq_before.append([pixloc2[j+2,:],h['DQ'].data[pixloc2[j+2,0],pixloc2[j+2,1]]])
        dq_before.append([pixloc2[j+3,:],h['DQ'].data[pixloc2[j+3,0],pixloc2[j+3,1]]])
            
    if save_figs == True:
                    
        # save figure of input dq vals
        fig, ax = plt.subplots(1,1,figsize=(10,10))
        plt.ylabel('y pixels',fontsize=15)
        plt.xlabel('x pixels',fontsize=15)
        plt.imshow((h['DQ'].data == 4.0), vmin=0, vmax=1, cmap=plt.cm.gray, origin='lower')
        ax.set_title("DQ Input"+str(i),fontsize=15)
        plt.colorbar(orientation='horizontal',pad=0.09)
        plt.savefig(outlier_json_file[:5]+str(i)+"_inputDQ.png")

### Get the after DQ pixel values(should be 17.0)

In [13]:
for i in range(len(file_list)):
    with fits.open(output_files[i]) as h:
        j = 4*i
        dq_after.append([pixloc2[j,:],h['DQ'].data[pixloc2[j,0],pixloc2[j,1]]])
        dq_after.append([pixloc2[j+1,:],h['DQ'].data[pixloc2[j+1,0],pixloc2[j+1,1]]])
        dq_after.append([pixloc2[j+2,:],h['DQ'].data[pixloc2[j+2,0],pixloc2[j+2,1]]])
        dq_after.append([pixloc2[j+3,:],h['DQ'].data[pixloc2[j+3,0],pixloc2[j+3,1]]])
        all_out_dqs.append((h['DQ'].data[pixloc2[j,0],pixloc2[j,1]] == 17.0))
        all_out_dqs.append((h['DQ'].data[pixloc2[j+1,0],pixloc2[j+1,1]] == 17.0))
        all_out_dqs.append((h['DQ'].data[pixloc2[j+2,0],pixloc2[j+2,1]] == 17.0))
        all_out_dqs.append((h['DQ'].data[pixloc2[j+3,0],pixloc2[j+3,1]] == 17.0))
                
    if save_figs == True:
                    
        # save figure of output dq values
        fig, ax = plt.subplots(1,1,figsize=(10,10))
        plt.ylabel('y pixels',fontsize=15)
        plt.xlabel('x pixels',fontsize=15)
        plt.imshow((h['DQ'].data == 4.0), vmin=0, vmax=1, cmap=plt.cm.gray, origin='lower')
        ax.set_title("DQ Output"+str(i),fontsize=15)
        plt.colorbar(orientation='horizontal',pad=0.09)
        plt.savefig(outlier_json_file[:5]+str(i)+"_outputDQ.png")

### Check if outlier_detection Pytest passed

In [14]:
print('Output DQ values: ',all_out_dqs)
assert np.alltrue(all_out_dqs) == True
print('MIRI Outlier Detetion Pytest: Passed')

Output DQ values:  [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
MIRI Outlier Detetion Pytest: Passed


<a id="about_ID"></a>
## About this Notebook
**Author:** Tea Temim
<br>**Updated On:** 08/06/20 to add in documentation